This file was our initial feature engineering attempt with the TotalSpending feature which resulted in extremely poor accuracies with the models which lead to us scrapping the feature

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('train.csv')
print(df.dtypes)
print(df.shape)
df.head()

PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object
(8693, 14)


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


after an initial glance at the data provided there are already a few things that i plan to do.

Firstly, I will change the cryosleep and vip to a binary result to ease the processing and limit miss interpretation for the machine learning model.

Secondly, I am thinking of deleting the names as they serve no true purpose for this data.

thirdly, i want to split the cabin feature into 3 features, one being deck, second is the number, and third is the side (which i will turn into a binary since there are only 2 sides)

Finally, I will number the home planets to make the machine learning model to understand it clearer.

All of these steps will lead to cleaner data visualizations and a more accurate model at the end.

But before proceed with any of the following steps I must first further understand the data the the relations between the features.

I have added a feature called TotalSpending that is the accumulation off all money spent on the cruise as that will be a lot more useful in relations and analysis than the individual spending categories

In [3]:
nl  = df.isnull().sum()
nl[nl>0]

HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
dtype: int64

After finding many null values in the data we must first remove them in order to clearly visualize the data

To solve the null values in RoomService FoodCourt, ShoppingMall, Spa, and VRDeck I will replace them with Zeros in order to not effect the Totalspending feature i am planning to add as they are all over the place with there numbers not being normalized

In [4]:
df['RoomService'] = df['RoomService'].fillna(0)
df['ShoppingMall'] = df['ShoppingMall'].fillna(0)
df['Spa'] = df['Spa'].fillna(0)
df['FoodCourt'] = df['FoodCourt'].fillna(0)
df['VRDeck'] = df['VRDeck'].fillna(0)

To solve the Null Values in CryoSleep, and VIP I will set all True values to 1 and other values to 0

In [5]:
df['CryoSleep'] = df['CryoSleep'].astype(str)
df['CryoSleep'] = df['CryoSleep'].map(lambda x: 1 if x=='True' else 0)

In [6]:
df['VIP'] = df['VIP'].astype(str)
df['VIP'] = df['VIP'].map(lambda x: 1 if x=='True' else 0)

The null Values HomePlanet, Destination, and Cabin will all be filled by Mode
and in cabin it will not really matter the repitition of room numbers as they will be deleted later we only need the deck and the side.
Mode null replacement for categorical data aids in not affecting the accuracy and the spread of the data.

In [7]:
df['HomePlanet']=  df['HomePlanet'].fillna(df['HomePlanet'].mode()[0])
df['Cabin']=  df['Cabin'].fillna(df['Cabin'].mode()[0])
df['Destination']=  df['Destination'].fillna(df['Destination'].mode()[0])

Finally the missing null values are in Name and Age. In Age i will put the median of the data in order to not affect it in a bais manner and in the name i will delete the column as it serves no purpose in pattern recognition

In [8]:
df['Age']=  df['Age'].fillna(df['Age'].mean())
del df['Name']


We have finally delt with all the Null values now it is time to expand upon out features such as Cabin and combining all our spending features into 1 column

In [9]:
new = df['Cabin'].astype(str).str.split('/', expand=True)
new = new.rename(columns={0:'Deck', 1:'RoomNumber', 2:'Side'})
df.insert(3, 'Side', new['Side'])
df.insert(3, 'Deck', new['Deck'])

Since there are only 2 sides to the cabin data (P or S) I have changed it in to a 1 or 0 instead to make processing easier for the ML model

In [10]:
df['Side'] = df['Side'].astype(str)
df['Side'] = df['Side'].map(lambda x: 1 if x=='S' else 0)

Then I combined all spending features into 1 total spendings and since i wont be needing the those columns i will delete them after combining then into a feature

In [11]:
df.insert(12,'TotalSpending', df['RoomService'] + df['FoodCourt'] + df['ShoppingMall'] + df['Spa'] + df['VRDeck'])
df.head()

,PassengerId,HomePlanet,CryoSleep,Deck,Side,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,TotalSpending,Spa,VRDeck,Transported
0,0001_01,Europa,0,B,0,B/0/P,TRAPPIST-1e,39.0,0,0.0,0.0,0.0,0.0,0.0,0.0,False
1,0002_01,Earth,0,F,1,F/0/S,TRAPPIST-1e,24.0,0,109.0,9.0,25.0,736.0,549.0,44.0,True
2,0003_01,Europa,0,A,1,A/0/S,TRAPPIST-1e,58.0,1,43.0,3576.0,0.0,10383.0,6715.0,49.0,False
3,0003_02,Europa,0,A,1,A/0/S,TRAPPIST-1e,33.0,0,0.0,1283.0,371.0,5176.0,3329.0,193.0,False
4,0004_01,Earth,0,F,1,F/1/S,TRAPPIST-1e,16.0,0,303.0,70.0,151.0,1091.0,565.0,2.0,True


Finally we deal with the ID by extracting from it only the group

In [12]:
new = df['PassengerId'].astype(str).str.split('_', expand=True)
df.insert(0, 'GroupId', new[0].astype(int))
df.head()

,GroupId,PassengerId,HomePlanet,CryoSleep,Deck,Side,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,TotalSpending,Spa,VRDeck,Transported
0,1,0001_01,Europa,0,B,0,B/0/P,TRAPPIST-1e,39.0,0,0.0,0.0,0.0,0.0,0.0,0.0,False
1,2,0002_01,Earth,0,F,1,F/0/S,TRAPPIST-1e,24.0,0,109.0,9.0,25.0,736.0,549.0,44.0,True
2,3,0003_01,Europa,0,A,1,A/0/S,TRAPPIST-1e,58.0,1,43.0,3576.0,0.0,10383.0,6715.0,49.0,False
3,3,0003_02,Europa,0,A,1,A/0/S,TRAPPIST-1e,33.0,0,0.0,1283.0,371.0,5176.0,3329.0,193.0,False
4,4,0004_01,Earth,0,F,1,F/1/S,TRAPPIST-1e,16.0,0,303.0,70.0,151.0,1091.0,565.0,2.0,True


In [13]:
del df['RoomService']
del df['FoodCourt']
del df['ShoppingMall']
del df['Spa']
del df['VRDeck']
del df['Cabin']
del df['PassengerId']
df.head()

,GroupId,HomePlanet,CryoSleep,Deck,Side,Destination,Age,VIP,TotalSpending,Transported
0,1,Europa,0,B,0,TRAPPIST-1e,39.0,0,0.0,False
1,2,Earth,0,F,1,TRAPPIST-1e,24.0,0,736.0,True
2,3,Europa,0,A,1,TRAPPIST-1e,58.0,1,10383.0,False
3,3,Europa,0,A,1,TRAPPIST-1e,33.0,0,5176.0,False
4,4,Earth,0,F,1,TRAPPIST-1e,16.0,0,1091.0,True


In [14]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df['HomePlanet'] = le.fit_transform(df.HomePlanet.values)
df['Destination'] = le.fit_transform(df.Destination.values)
df['Deck'] = le.fit_transform(df.Deck.values)
df.head()

,GroupId,HomePlanet,CryoSleep,Deck,Side,Destination,Age,VIP,TotalSpending,Transported
0,1,1,0,1,0,2,39.0,0,0.0,False
1,2,0,0,5,1,2,24.0,0,736.0,True
2,3,1,0,0,1,2,58.0,1,10383.0,False
3,3,1,0,0,1,2,33.0,0,5176.0,False
4,4,0,0,5,1,2,16.0,0,1091.0,True


In [15]:
transported = df['Transported']
del df['Transported']
from sklearn.model_selection import train_test_split
x_train, x_validate, y_train, y_validate = train_test_split(df, transported,train_size= 0.75, test_size= 0.25, random_state=42)

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
print('Linear Regression model')
lr = LogisticRegression(max_iter=10000)
lr.fit(x_train, y_train)
predicted = lr.predict(x_validate)
acc = metrics.accuracy_score(y_validate,predicted)
print ('accuracy = '+str(acc*100)+'%')
print (metrics.classification_report(y_validate,predicted))
print("Confusion Matrix")
print(metrics.confusion_matrix(y_validate,predicted))

Linear Regression model
accuracy = 71.8951241950322%
              precision    recall  f1-score   support

       False       0.69      0.80      0.74      1082
        True       0.77      0.64      0.69      1092

    accuracy                           0.72      2174
   macro avg       0.73      0.72      0.72      2174
weighted avg       0.73      0.72      0.72      2174

Confusion Matrix
[[869 213]
 [398 694]]


In [17]:
from sklearn.tree import DecisionTreeClassifier
print("ID3")
clf = DecisionTreeClassifier()
clf.fit(x_train, y_train)
predicted = clf.predict(x_validate)
acc = metrics.accuracy_score(y_validate,predicted)
print ('accuracy = '+str(acc*100)+'%')
print (metrics.classification_report(y_validate,predicted))
print("Confusion Matrix")
print(metrics.confusion_matrix(y_validate,predicted))

ID3
accuracy = 68.12327506899723%
              precision    recall  f1-score   support

       False       0.69      0.65      0.67      1082
        True       0.67      0.71      0.69      1092

    accuracy                           0.68      2174
   macro avg       0.68      0.68      0.68      2174
weighted avg       0.68      0.68      0.68      2174

Confusion Matrix
[[706 376]
 [317 775]]


In [18]:
from sklearn.ensemble import RandomForestClassifier
print("Random Forests")
rf = RandomForestClassifier(criterion='log_loss')
rf.fit(x_train, y_train)
predicted = rf.predict(x_validate)
acc = metrics.accuracy_score(y_validate,predicted)
print ('accuracy = '+str(acc*100)+'%')
print (metrics.classification_report(y_validate,predicted))
print("Confusion Matrix")
print(metrics.confusion_matrix(y_validate,predicted))

Random Forests
accuracy = 73.78104875804968%
              precision    recall  f1-score   support

       False       0.71      0.80      0.75      1082
        True       0.77      0.68      0.72      1092

    accuracy                           0.74      2174
   macro avg       0.74      0.74      0.74      2174
weighted avg       0.74      0.74      0.74      2174

Confusion Matrix
[[861 221]
 [349 743]]


In [19]:
from sklearn import naive_bayes
print('Naive Bayes')
clfr = naive_bayes.MultinomialNB()
clfr.fit(x_train, y_train)
predicted = clfr.predict(x_validate)
acc = metrics.accuracy_score(y_validate,predicted)
print ('accuracy = '+str(acc*100)+'%')
print (metrics.classification_report(y_validate,predicted))
print("Confusion Matrix")
print(metrics.confusion_matrix(y_validate,predicted))

Naive Bayes
accuracy = 64.58141674333027%
              precision    recall  f1-score   support

       False       0.72      0.47      0.57      1082
        True       0.61      0.82      0.70      1092

    accuracy                           0.65      2174
   macro avg       0.67      0.64      0.63      2174
weighted avg       0.67      0.65      0.63      2174

Confusion Matrix
[[506 576]
 [194 898]]


In [20]:
from sklearn.neighbors import KNeighborsClassifier
print('KNN')
knn = KNeighborsClassifier(n_neighbors=31)
knn.fit(x_train,y_train)
predicted = knn.predict(x_validate)
acc = metrics.accuracy_score(y_validate,predicted)
print ('accuracy = '+str(acc*100)+'%')
print (metrics.classification_report(y_validate,predicted))
print("Confusion Matrix")
print(metrics.confusion_matrix(y_validate,predicted))

KNN
accuracy = 73.41306347746091%
              precision    recall  f1-score   support

       False       0.71      0.78      0.74      1082
        True       0.76      0.69      0.72      1092

    accuracy                           0.73      2174
   macro avg       0.74      0.73      0.73      2174
weighted avg       0.74      0.73      0.73      2174

Confusion Matrix
[[843 239]
 [339 753]]


In [21]:
from sklearn import svm
print('SVM')
clf = svm.SVC(kernel='linear')
clf.fit(x_train, y_train)
predicted = clf.predict(x_validate)
acc = metrics.accuracy_score(y_validate,predicted)
print ('accuracy = '+str(acc*100)+'%')
print (metrics.classification_report(y_validate,predicted))
print("Confusion Matrix")
print(metrics.confusion_matrix(y_validate,predicted))


SVM
accuracy = 71.20515179392825%
              precision    recall  f1-score   support

       False       0.69      0.78      0.73      1082
        True       0.75      0.65      0.69      1092

    accuracy                           0.71      2174
   macro avg       0.72      0.71      0.71      2174
weighted avg       0.72      0.71      0.71      2174

Confusion Matrix
[[843 239]
 [387 705]]
